In [20]:
import os
os.chmod('./img', 755)             # 파일의 사용 권한 변경

import urllib.request             # 웹 url 인식
from  bs4 import BeautifulSoup   # html에서 데이터 검색에 활용
from selenium import webdriver   # 웹 드라이버 자동화 library
from selenium.webdriver.common.keys import Keys  # 클릭 자동화
import time                      # sleep을 걸기 위한 time library


# URL 받아오기
browser = webdriver.Chrome('chromedriver')    # 브라우저 인스턴스화

# 구글의 이미지 검색 url
browser.get("https://www.google.co.kr/imghp?hl=ko&tab=ri&ogbl")

# 구글의 이미지 검색에 해당하는 input 창(검색창에 해당하는 html코드를 찾아서 element 사용)
elem = browser.find_element_by_xpath("//*[@class='gLFyf gsfi']") 


# 검색어 입력
# 사용한 키워드 5가지: '스쿼트 자세', 'how to squat', 'squat angle', 'squat basic', 'learn squat'
elem.send_keys("learn squat")

# 웹 내 submit : enter(역할)
elem.submit()

# 스크롤을 내림 : 브라우져 이미지 검색결과 부분(바디부분)에 마우스 클릭 한번 하고 End키를 누름
for i in range(1, 20):   # 20회 반복 -> 한 번에 400개의 이미지 다운로드
    browser.find_element_by_xpath("//body").send_keys(Keys.END)
    time.sleep(10)                  # END 키 누르고 내려가는데 시간이 걸려서 sleep

time.sleep(10)                      # 안정성 위해 sleep
html = browser.page_source         # 크롬브라우져에서 현재 불러온 소스 가져옴
soup = BeautifulSoup(html, "lxml") # html 코드를 검색할 수 있도록 설정

# 이미지 url list 
def fetch_list_url():
    params = []
    imgList = soup.find_all("img", class_="rg_i Q4LuWd")  # 구글 이미지 url 내 img 태그의 _img 클래스
    for im in imgList:
        try :
            params.append(im["src"])                   # params 리스트에 image url 을 담음
        except KeyError:                              # 에러 시 data-src(데이터의 의미 없음)
            params.append(im["data-src"])     
    return params

# 이미지 다운로드
def fetch_detail_url():
    params = fetch_list_url()
    print(params)
    for idx,p in enumerate(params,1):  # enumerate : 리스트의 모든 요소를 index, pair로
        # 다운받을 폴더경로 입력
        urllib.request.urlretrieve(p, "./img" + str(idx) + ".jpg") # 파일명: img+숫자

fetch_detail_url()

# 완료 시 브라우저 닫기
browser.quit()


In [30]:
# human(object) detection & Crop

import cv2         # 이미지를 다루는 library

# 이미지 crop
def imagecrop(image):
    hog= cv2.HOGDescriptor() 
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())  # HOG기반 SVM을 활용한 객체 추정 모델 
    hogParms={'winStride':(5,5),'padding':(16,16),'scale':1.05, 'hitThreshold':0,'finalThreshold':5}
    # Stride : 5, padding : 16으로 설정, threshold: 임계점(기준점) - 작을수록 엄격하게 검출
    
    img = cv2.imread(image)  # 이미지 읽기
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)       # gray color
    human,r= hog.detectMultiScale(gray,**hogParms)    # detect 수행, human / hog parameter 저장
    
    if(len(human)>0):
        for(x,y,w,h) in human:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),3)   # detect된 human의 좌표(x, y, h ,w)를 불러와서 사각형을 그리고
            sub_human = img[y:y+h, x:x+w]                        # 이미지 잘라내기
            fname, ext = os.path.splitext(image)                 # 크롭된 이미지 이름, extraction 
            cv2.imwrite(fname+"_cropped_"+ext, sub_human)        # 크롭된 이미지 저장      

In [31]:
data_path = './dataset_crawling/'    # 저장할 경로 지정
file_list = os.listdir(data_path)    # 폴더 내 파일명을 리스트로 저장

for i in range(len(file_list)):      # imagecrop 메소드 반복 수행
    imagecrop('./dataset_crawling/'+file_list[i])